In [1]:
# Environment Setup
# ----------------------------------------------------------------
# Dependencies
import csv
import pandas as pd
import random
import numpy as np
import string

# Output File Names
file_output_clinical_trial_csv = "generated_data/clinicaltrial_data.csv"
file_output_mouse_drug_csv = "generated_data/mouse_drug_data.csv"

# Environment Setup
# ----------------------------------------------------------------
# Dependencies
import csv
import pandas as pd
import random
import numpy as np
import string

# Output File Names
file_output_clinical_trial_csv = "generated_data/clinicaltrial_data.csv"
file_output_mouse_drug_csv = "generated_data/mouse_drug_data.csv"

# Generator Conditions (Change as Needed)
# ----------------------------------------------------------------

# Time Points
time_points = [5, 10, 15, 20, 25, 30, 35, 40, 45]

# Drug Options
drug_names = ["Stelasyn", "Naftisol", "Ketapril", "Capomulin",
              "Infubinol", "Ceftamin", "Propriva", "Zoniferol", 
              "Ramicane", "Placebo"]

# Top Drug Count
top_drug_count = 2

# Mice Names
mouse_count = 25

# Tumor Volume Range
tumor_volume_range = (45, 400)

# Metastatic Range
met_range = (0, 5)

# Tumor Volume Change (Change and Liklihood)
cure_rates = [((-0.15, -0.01), 45), ((0.01, 0.02), 55)]
fail_rates = [((0.01, 0.05), 65), ((0.05, 0.10), 35)]

# Met Change (Change and Liklihood)
cure_met_rates = [(0, 85), (1, 15)]
fail_met_rates = [(0, 70), (1, 30)] 

# Mortality Rate
low_death_rates = [("Death", 4), ("Live", 96)]
high_death_rates = [("Death", 9), ("Live", 91)]

In [2]:
# Select "Top Drugs" at Random
# ----------------------------------------------------------------

# Top Drugs
top_drugs = random.sample(drug_names, top_drug_count)
bottom_drugs = list(set(drug_names) - set(top_drugs))


In [3]:
# Generate Mice
# ----------------------------------------------------------------

mouse_array = []
for drug in drug_names:
    for mouse in range(mouse_count):

        # Generate Mice IDs (eg: "e123")
        mouse_id = random.choice(string.ascii_lowercase) + str(random.randint(100, 999))
        
        # Associate the mice with specific drugs
        mouse_array.append([mouse_id, drug])

# Convert to Data Frame 
mouse_drug_pd = pd.DataFrame(mouse_array)
mouse_drug_pd.columns = ["Mouse ID", "Drug"] 

# Export the Mouse Drug Data
mouse_drug_pd.to_csv(file_output_mouse_drug_csv, index=False)

# Display Data
mouse_drug_pd.head()

,Mouse ID,Drug
0,x903,Stelasyn
1,u281,Stelasyn
2,w929,Stelasyn
3,a189,Stelasyn
4,y587,Stelasyn


In [4]:
# Determine which mice are in the "top drug" group
# ----------------------------------------------------------------

safe_mice = []
risk_mice = []

# Add mice to the "safe_mice" or "risk_mice" groups based on drug
for drug in drug_names:
    mice_list = mouse_drug_pd[(mouse_drug_pd.Drug == drug)]["Mouse ID"]
    
    mice_list = list(mice_list.values.flatten())
    
    if drug in top_drugs:
        safe_mice = safe_mice + mice_list 
    else:
        risk_mice = risk_mice + mice_list 

In [5]:
# Generate the Cancer Data (Tumor Volume)
# ----------------------------------------------------------------

# Empty list to hold all cancer data per mouse
mouse_cancer_data = []
mice_list = (safe_mice, risk_mice)

for i, mice in enumerate(mice_list):
    
    for mouse in mice:
        
        # Establish initial stats to be used in determining change
        pre_tumor_volume = tumor_volume_range[0]
        pre_met_count = met_range[0]
        new_tumor_volume = tumor_volume_range[0]
        new_met_count = met_range[0]
        
        # If safe mice then set the conditions appropriately
        if (i==0):
            time_outcome_vol = [cure_rates[0][0]] * cure_rates[0][1] + [cure_rates[1][0]] * cure_rates[1][1]
            time_outcome_met = [cure_met_rates[0][0]] * cure_met_rates[0][1] + [cure_met_rates[1][0]] * cure_met_rates[1][1]
            death_outcome = [low_death_rates[0][0]] * low_death_rates[0][1] + [low_death_rates[1][0]] * low_death_rates[1][1]
        
        # If risk mice then set the conditions appropriately
        else:
            time_outcome_vol = [fail_rates[0][0]] * fail_rates[0][1] + [fail_rates[1][0]] * fail_rates[1][1]
            time_outcome_met = [fail_met_rates[0][0]] * fail_met_rates[0][1] + [fail_met_rates[1][0]] * fail_met_rates[1][1]
            death_outcome = [high_death_rates[0][0]] * high_death_rates[0][1] + [high_death_rates[1][0]] * high_death_rates[1][1]

        # Initialize the mouse data
        mouse_cancer_data.append([mouse, 0, pre_tumor_volume, 0])
        
        # Loop through every time point and assign a tumor volume, met count, and death outcome.
        for time in time_points:

            # Determine if mouse dies... if so, quit adding rows
            if(random.choice(death_outcome) == "Death"):
                break

            # Select at random a time outcome
            time_outcome_datapoint = random.choice(time_outcome_vol) 
            change_rate = random.uniform(time_outcome_datapoint[0], time_outcome_datapoint[1])   

            # Select a random met outcome
            time_outcome_met_datapoint = random.choice(time_outcome_met)

            # Determine new tumor volume (Original + Growth * Original)
            new_tumor_volume = pre_tumor_volume + (change_rate * pre_tumor_volume)

            # Determine new met count (Original + New)
            if(pre_met_count < 4):
                new_met_count = pre_met_count + time_outcome_met_datapoint

            # Set the pre_tumor_volume equal to the most recent tumor volume
            pre_tumor_volume = new_tumor_volume

            # Set the new met count equal to the old met count
            pre_met_count = new_met_count

            # Add the entry to the mouse_cancer_data list
            mouse_cancer_data.append([mouse, time, new_tumor_volume, new_met_count])

# Convert Mouse Cancer Data to Data Frame
mouse_cancer_data = pd.DataFrame(mouse_cancer_data)
mouse_cancer_data.columns = ["Mouse ID", "Timepoint", "Tumor Volume (mm3)", "Metastatic Sites"]

# Sort the data based on Timepoint
mouse_cancer_data.sort_values("Timepoint", inplace=True)

# Export the Mouse Clinical Trial Data
mouse_cancer_data.to_csv(file_output_clinical_trial_csv, index=False)

# Display data
mouse_cancer_data.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,x903,0,45.0,0
720,h124,0,45.0,0
1678,p849,0,45.0,0
710,l726,0,45.0,0
700,a960,0,45.0,0
